In [3]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [4]:
!pip install PyAudio

In [5]:
import pyaudio
import os
import wave
import librosa
import numpy as np
import tensorflow as tf
import pandas as pd
import tqdm

In [6]:
from sys import byteorder
from array import array
from struct import pack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split

Loading data

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [7]:
# Read the dataset from a CSV file
dataset = pd.read_csv("data.csv")

# Determine the number of total, male, and female samples in the dataset
total_samples = len(dataset)
male_count = len(dataset[dataset['gender'] == 1])
female_count = len(dataset[dataset['gender'] == 0])

# Display the counts
print("Total number of samples:", total_samples)
print("Number of male samples:", male_count)
print("Number of female samples:", female_count)

# Create an array to store 128 audio features for each sample
X = np.zeros((total_samples, 128))

# Create an array to store gender labels for each sample (1 for male, 0 for female)
y = np.zeros((total_samples, 1))

# Loop through each sample to extract features and labels
for i, (file_name, gender_label) in tqdm.tqdm(enumerate(zip(dataset['filename'], dataset['gender'])), "Loading data", total=total_samples):
    #feature_data = np.load("/content/gdrive/My Drive/" + file_name)
    feature_data = np.load(file_name)
    X[i] = feature_data
    y[i] = gender_label

# Preprocess the feature data for neural network input
X = tf.keras.preprocessing.sequence.pad_sequences(X , dtype=np.float32, maxlen=144, padding='post')
X = X.reshape(-1, 12, 12)
X = np.expand_dims(X, axis=3)

Total number of samples: 66938
Number of male samples: 33469
Number of female samples: 33469


Loading data: 100%|██████████| 66938/66938 [00:12<00:00, 5562.31it/s]


In [11]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Further split the training set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=7)

# Flatten each sequence in the training features
X_flat = np.array([sequence.flatten() for sequence in X_train])

# Display the first 10 flattened sequences and their shape
#print(X_flat[:10])
#print(X_flat.shape)

# Convert flattened features to a DataFrame and drop the last 16 columns
# that we added to make our records 2D from 1D
X_flat = pd.DataFrame(X_flat)
X_flat = X_flat.iloc[:, :-16]
print(X_flat.shape)

# Train a logistic regression model on the processed training data
logreg = LogisticRegression(random_state=87, max_iter=100000).fit(X_flat, y_train.squeeze())

[[1.7404141e-02 9.1459781e-02 3.6782688e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.3867648e-04 1.0894923e-04 1.4831129e-04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.9285662e-02 3.1437859e-02 7.6510203e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [6.2559600e-05 1.2894056e-04 1.7443823e-03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.3852437e-02 2.2089299e-02 3.6699757e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [7.8245113e-03 7.5188670e-03 2.2727162e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
(48195, 144)
(48195, 128)


In [19]:
# Flatten each sequence in the test features
X_test_flat = np.array([sequence.flatten() for sequence in X_test])
X_test_flat = pd.DataFrame(X_test_flat)
X_test_flat = X_test_flat.iloc[:, :-16]

# Predict the labels for the test set
y_predtest = logreg.predict(X_test_flat)

# Calculate the accuracy of the model on the test set
accuracy_test = accuracy_score(y_test, y_predtest)
print(accuracy_test)

# Display the accuracy of the model on both the training and test sets
print("Training set accuracy: {:.4f}".format(logreg.score(X_flat, y_train.squeeze())))
print("Test set accuracy: {:.4f}".format(logreg.score(X_test_flat, y_test)))

0.8001195100089632
Training set accuracy: 0.7945
Test set accuracy: 0.8001


**CNN 2D**

In [21]:
# using binary crossentropy as it's male/female classification (binary)
inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]), batch_size = 64)

x = tf.keras.layers.Conv2D(128, 3, activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(256, 2, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(512, 1, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
# print summary of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(64, 12, 12, 1)]         0         
                                                                 
 conv2d (Conv2D)             (64, 10, 10, 128)         1280      
                                                                 
 max_pooling2d (MaxPooling2  (64, 5, 5, 128)           0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (64, 4, 4, 256)           131328    
                                                                 
 max_pooling2d_1 (MaxPoolin  (64, 2, 2, 256)           0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (64, 2, 2, 512)           131584

In [25]:
batch_size = 64
epochs = 100

# train the model using the training set and validating using validation set
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))

# save the model to a file

#model.save("results/model.h5")  

# evaluating the model using the testing set
print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
754/754 [==============================] - 6s 8ms/step - loss: 0.0332 - accuracy: 0.9912 - val_loss: 0.5809 - val_accuracy: 0.9246
Epoch 2/100
754/754 [==============================] - 6s 8ms/step - loss: 0.0194 - accuracy: 0.9930 - val_loss: 0.6280 - val_accuracy: 0.9268
Epoch 3/100
754/754 [==============================] - 6s 8ms/step - loss: 0.0117 - accuracy: 0.9952 - val_loss: 0.6379 - val_accuracy: 0.9313
Epoch 4/100
754/754 [==============================] - 7s 9ms/step - loss: 0.0282 - accuracy: 0.9918 - val_loss: 0.6935 - val_accuracy: 0.9281
Epoch 5/100
754/754 [==============================] - 6s 8ms/step - loss: 0.0301 - accuracy: 0.9914 - val_loss: 0.6159 - val_accuracy: 0.9283
Epoch 6/100
754/754 [==============================] - 6s 8ms/step - loss: 0.0145 - accuracy: 0.9950 - val_loss: 0.7806 - val_accuracy: 0.9203
Epoch 7/100
754/754 [==============================] - 6s 8ms/step - loss: 0.0190 - accuracy: 0.9938 - val_loss: 0.6387 - val_accuracy: 0.9292

In [26]:
# using binary crossentropy as it's male/female classification (binary)
inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]), batch_size = 64)

x2 = tf.keras.layers.Conv2D(64, 3, activation='relu')(inputs)
x2 = tf.keras.layers.MaxPooling2D()(x2)

x2 = tf.keras.layers.Conv2D(128, 2, activation='relu')(x2)
x2 = tf.keras.layers.MaxPooling2D()(x2)

x2 = tf.keras.layers.Conv2D(256, 1, activation='relu')(x2)
x2 = tf.keras.layers.MaxPooling2D()(x2)

x2 = tf.keras.layers.Flatten()(x2)

x2 = tf.keras.layers.Dense(512, activation='relu')(x2)

outputs2 = tf.keras.layers.Dense(1, activation='sigmoid')(x2)

model2 = tf.keras.Model(inputs, outputs2)
model2.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
# print summary of the model
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(64, 12, 12, 1)]         0         
                                                                 
 conv2d_3 (Conv2D)           (64, 10, 10, 64)          640       
                                                                 
 max_pooling2d_3 (MaxPoolin  (64, 5, 5, 64)            0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (64, 4, 4, 128)           32896     
                                                                 
 max_pooling2d_4 (MaxPoolin  (64, 2, 2, 128)           0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (64, 2, 2, 256)           3302

In [27]:
batch_size = 64
epochs = 100

# train the model using the training set and validating using validation set
model2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))

# save the model to a file

#model.save("results/model.h5")  

# evaluating the model using the testing set
print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
754/754 [==============================] - 3s 4ms/step - loss: 0.4571 - accuracy: 0.7870 - val_loss: 0.3424 - val_accuracy: 0.8571
Epoch 2/100
754/754 [==============================] - 3s 5ms/step - loss: 0.3350 - accuracy: 0.8580 - val_loss: 0.2937 - val_accuracy: 0.8829
Epoch 3/100
754/754 [==============================] - 4s 5ms/step - loss: 0.2965 - accuracy: 0.8777 - val_loss: 0.3594 - val_accuracy: 0.8525
Epoch 4/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2788 - accuracy: 0.8860 - val_loss: 0.2715 - val_accuracy: 0.8835
Epoch 5/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2532 - accuracy: 0.8965 - val_loss: 0.2526 - val_accuracy: 0.8967
Epoch 6/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2390 - accuracy: 0.9036 - val_loss: 0.2247 - val_accuracy: 0.9092
Epoch 7/100
754/754 [==============================] - 3s 4ms/step - loss: 0.2271 - accuracy: 0.9078 - val_loss: 0.2654 - val_accuracy: 0.8952

In [33]:
# using binary crossentropy as it's male/female classification (binary)
inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]), batch_size = 64)

x3 = tf.keras.layers.Conv2D(64, 3, activation='relu')(inputs)
x3 = tf.keras.layers.MaxPooling2D()(x3)

x3 = tf.keras.layers.Conv2D(128, 2, activation='relu')(x3)
x3 = tf.keras.layers.MaxPooling2D()(x3)

x3 = tf.keras.layers.Flatten()(x3)

x3 = tf.keras.layers.Dense(256, activation='relu')(x3)

outputs3 = tf.keras.layers.Dense(1, activation='sigmoid')(x3)

model3 = tf.keras.Model(inputs, outputs3)
model3.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
# print summary of the model
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(64, 12, 12, 1)]         0         
                                                                 
 conv2d_3 (Conv2D)           (64, 10, 10, 64)          640       
                                                                 
 max_pooling2d_3 (MaxPoolin  (64, 5, 5, 64)            0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (64, 4, 4, 128)           32896     
                                                                 
 max_pooling2d_4 (MaxPoolin  (64, 2, 2, 128)           0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (64, 2, 2, 256)           3302

In [34]:
batch_size = 64
epochs = 100

# train the model using the training set and validating using validation set
model3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))

# save the model to a file

model3.save("results/model.h5")  

# evaluating the model using the testing set
print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model3.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Epoch 1/100
754/754 [==============================] - 3s 4ms/step - loss: 0.4522 - accuracy: 0.7984 - val_loss: 0.3482 - val_accuracy: 0.8596
Epoch 2/100
754/754 [==============================] - 2s 3ms/step - loss: 0.3237 - accuracy: 0.8610 - val_loss: 0.2957 - val_accuracy: 0.8766
Epoch 3/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2854 - accuracy: 0.8809 - val_loss: 0.2754 - val_accuracy: 0.8915
Epoch 4/100
754/754 [==============================] - 3s 3ms/step - loss: 0.2574 - accuracy: 0.8948 - val_loss: 0.2370 - val_accuracy: 0.9120
Epoch 5/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2370 - accuracy: 0.9039 - val_loss: 0.2384 - val_accuracy: 0.9053
Epoch 6/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2239 - accuracy: 0.9113 - val_loss: 0.2140 - val_accuracy: 0.9195
Epoch 7/100
754/754 [==============================] - 2s 3ms/step - loss: 0.2110 - accuracy: 0.9170 - val_loss: 0.2052 - val_accuracy: 0.9210

**Demo**

In [38]:
pip install wavio

Note: you may need to restart the kernel to use updated packages.


In [39]:
import sounddevice as sd
import wavio
import scipy.stats

In [46]:
# def record_audio(duration=5, filename='recorded_voice.wav', fs=44100):
#     print("Recording...")
#     audio = sd.rec(int(duration * fs), samplerate=fs, channels=2)
#     sd.wait()  # Wait until recording is finished
#     print("Recording finished. Saving file...")
#     wavio.write(filename, audio, fs, sampwidth=2)  # Save as WAV file
#     return filename

import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write

def list_devices():
    """
    List all available audio input and output devices.
    """
    print(sd.query_devices())

def record_audio(duration=5, fs=44100, channels=1, filename='recorded_voice.wav'):
    """
    Record audio for a given duration, sample rate, and number of channels.
    
    :param duration: Duration in seconds
    :param fs: Sampling rate
    :param channels: Number of audio channels
    :return: NumPy array of recorded audio
    """
    try:
        print("Recording...")
        recording = sd.rec(int(duration * fs), samplerate=fs, channels=channels)
        sd.wait()  # Wait until recording is finished
        print("Recording complete")
        print("Recording finished. Saving file...")
        wavio.write(filename, recording, fs, sampwidth=2)
        return recording
    except Exception as e:
        print(f"An error occurred: {e}")

# List available devices
list_devices()

  0 Rakhat’s iPhone Microphone, Core Audio (1 in, 0 out)
> 1 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 2 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  3 BoomAudio, Core Audio (6 in, 6 out)
  4 Microsoft Teams Audio, Core Audio (2 in, 2 out)


In [47]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    #result = np.hstack((result, np.zeros(1, 16)))
    for i in range(16):
        result = np.hstack((result, 0))
    #print(result.shape)
    #print(result)
    return result

In [1]:
pip install ffmpeg

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Example usage - try changing channels if needed
audio_data = record_audio(duration=5, fs=44100, channels=1)
features = extract_feature(file, mel=True)
    
features = features.reshape(-1, 12, 12)
#print(features)
features = np.expand_dims(features, axis=3)
#print(model.predict(features))
male_prob = model3.predict(features)[0][0]
female_prob = 1 - male_prob
gender = "male" if male_prob > female_prob else "female"
# show the result!
print("Result:", gender)


NameError: name 'record_audio' is not defined

Loading data

In [36]:
THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.
    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()



def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    #result = np.hstack((result, np.zeros(1, 16)))
    for i in range(16):
        result = np.hstack((result, 0))
    #print(result.shape)
    #print(result)
    return result


if __name__ == "__main__":
    # load the saved model (after training)
    # model = pickle.load(open("result/mlp_classifier.model", "rb"))
    import argparse
    parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
    parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
    args = parser.parse_args()
    file = args.file
    #model3.load_weights("results/model.h5")
    # construct the model
    # load the saved/trained weight
    #print(model)
    if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        print("Please talk")
        # put the file name here
        file = "test.wav"
        # record the file (start talking)
        record_to_file(file)
    # extract features and reshape it
    features = extract_feature(file, mel=True)
    
    features = features.reshape(-1, 12, 12)
    #print(features)
    features = np.expand_dims(features, axis=3)
    #print(model.predict(features))
    male_prob = model3.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    print("Result:", gender)
    ##print(f"Probabilities:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")


/var/folders/1_/pw0_qvk92kxcltkjvbtqf73w0000gn/T/ipykernel_29818/3010357707.py:130: UserWarning: PySoundFile failed. Trying audioread instead.
  X, sample_rate = librosa.core.load(file_name)


NoBackendError: 